<a href="https://colab.research.google.com/github/RobertoDelGiudice/MovieGenresClassification/blob/main/Data_tidying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the very first code of my project:

In this one script our goal is obtaining a cleaned dataset for the further analysis

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now, let's upload our dataset and drop every *Null* value

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/IMDb movies.csv")
data = data[['imdb_title_id','title','genre','description']]
data = data.dropna()
data

<ipython-input-1-a639087d9332>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/IMDb movies.csv")


,imdb_title_id,title,genre,description
0,tt0000009,Miss Jerry,Romance,The adventures of a female reporter in the 1890s.
1,tt0000574,The Story of the Kelly Gang,"Biography, Crime, Drama",True story of notorious Australian outlaw Ned ...
2,tt0001892,Den sorte drøm,Drama,Two men of high rank are both wooing the beaut...
3,tt0002101,Cleopatra,"Drama, History",The fabled queen of Egypt's affair with Roman ...
4,tt0002130,L'Inferno,"Adventure, Drama, Fantasy",Loosely adapted from Dante's Divine Comedy and...
...,...,...,...,...
85848,tt9905462,Pengalila,Drama,An unusual bond between a sixty year old Dalit...
85849,tt9906644,Manoharam,"Comedy, Drama",Manoharan is a poster artist struggling to fin...
85850,tt9908390,Le lion,Comedy,A psychiatric hospital patient pretends to be ...
85851,tt9911196,De Beentjes van Sint-Hildegard,"Comedy, Drama",A middle-aged veterinary surgeon believes his ...


We need to split our *Genre* column in three separated columns, because there are movies with at maximum of three genres.
After that we need to get a longer table with each genre as a new row and only one column (instead of three) that we will call *value*.
Then we drop *Null* values again.

In [ ]:
data[['0','1','2']] = data['genre'].str.split(', ', expand=True)
data1 = pd.melt(data, id_vars=['imdb_title_id','title','description'], value_vars=['0','1','2'])
data1 = data1.dropna()

Some plots have less than 20 characters, so they could mislead our embedding encoding process. I suggest to eliminate these movies.

In [ ]:
short_plot = [word for word in data1.description.unique() if len(word) <= 20]

In [ ]:
data1 = data1[~data1["description"].isin(short_plot)]
data1 = data1.dropna()

Now, let's find the movies to be dropped, because contain other genres outside our classificator scope.

In [ ]:
stop = data1["imdb_title_id"][~data1["value"].isin(["Drama","Comedy","Action","Horror"])]

In [ ]:
data3 = data1[~data1["imdb_title_id"].isin(stop)]
data3=data3.rename(columns={"value":"Genre","variable":"value"})
data3["value"]=1

Now let's aggregate our Genre in a 4-elements binary array and save the entire dataset.

In [ ]:
import numpy as np
data4 = pd.pivot(data3, index=["imdb_title_id","title","description"], columns="Genre", values="value")
data4 = data4.fillna(0)
data4 = data4.reset_index()
data4.columns.name = None
data4['Genre'] = data4.apply(lambda row: np.array(row.values[3:]), axis=1)
data4
data4.to_pickle("/content/drive/MyDrive/dataframe.pkl")

,imdb_title_id,title,description,Action,Comedy,Drama,Horror,Genre
0,tt0001892,Den sorte drøm,Two men of high rank are both wooing the beaut...,0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
1,tt0002461,Richard III,Richard of Gloucester uses manipulation and mu...,0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
2,tt0002646,Atlantis,After Dr. Friedrich's wife becomes mentally un...,0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
3,tt0003014,Il calvario di una madre,Single mother is separated from her children d...,0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
4,tt0003102,Ma l'amor mio non muore...,"Leslie Swayne, an adventurer, in order to obta...",0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...
28991,tt9905462,Pengalila,An unusual bond between a sixty year old Dalit...,0.0,0.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0]"
28992,tt9906644,Manoharam,Manoharan is a poster artist struggling to fin...,0.0,1.0,1.0,0.0,"[0.0, 1.0, 1.0, 0.0]"
28993,tt9908390,Le lion,A psychiatric hospital patient pretends to be ...,0.0,1.0,0.0,0.0,"[0.0, 1.0, 0.0, 0.0]"
28994,tt9911196,De Beentjes van Sint-Hildegard,A middle-aged veterinary surgeon believes his ...,0.0,1.0,1.0,0.0,"[0.0, 1.0, 1.0, 0.0]"


That's all! Our dataset has been tidied and is now ready for further analysis.



---

**The following code has been written to obtain a first overview of our final dataset**

In [ ]:
data2 = data1[~data1["imdb_title_id"].isin(stop)]
data2 = pd.pivot(data2, index=["imdb_title_id","title","description"], columns="variable", values="value")
data2 = data2.fillna('')
data2['Genre'] = data2.apply(lambda row: [row['0'], row['1'], row['2']], axis=1)
data2['Genre'] = data2['Genre'].apply(lambda x: sorted(x))
data2['Genre'] = data2['Genre'].apply(lambda x: ''.join(x))
count = data2.groupby(['Genre'])['Genre'].count()
sorted_counts = count.sort_values(ascending=False)
total=pd.Series({'Total':sorted_counts.sum()}, name='Total')
sorted_counts=pd.concat([sorted_counts,total])

In [ ]:
sorted_counts

Drama                 12073
Comedy                 7123
ComedyDrama            3999
Horror                 2240
ActionDrama             886
Action                  699
ComedyHorror            579
ActionComedy            464
ActionComedyDrama       322
DramaHorror             232
ActionComedyHorror      138
ActionHorror             86
ActionDramaHorror        82
ComedyDramaHorror        73
Total                 28996
dtype: int64